In [1]:
from plugins.battle.models import Player, get_player, sign_reply
from event import fake_group_msg_event

player = get_player(user_id=435786117, group_id=542423773)
print(player.status())

[CQ:at,qq=435786117](21级)当前状态: 
升级所需经验: 200
生命: 200/200 体力: 10/10
攻击: 45/90 K/D: 0/4
黄金: 3591



In [3]:
from plugins.battle import CheckStatusPlugin

event = fake_group_msg_event('状态')
plugin = CheckStatusPlugin(event)
reply = plugin.get_reply()

# self.assertIn('升级所需经验', reply)
# self.assertIn('生命', reply)

In [7]:
player = get_player(plugin.user_id, plugin.group_id)

In [9]:
Player.get(user_id=plugin.user_id, group_id=plugin.group_id)

PlayerDoesNotExist: <Model: Player> instance matching query does not exist:
SQL: SELECT `t1`.`id`, `t1`.`user_id`, `t1`.`group_id`, `t1`.`dead_time`, `t1`.`gold`, `t1`.`num_sign`, `t1`.`last_sign`, `t1`.`q1_weapon`, `t1`.`q5_weapon`, `t1`.`weapon_limit`, `t1`.`weapon_limit_max`, `t1`.`q1_food`, `t1`.`q5_food`, `t1`.`food_limit`, `t1`.`food_limit_max`, `t1`.`limit_refresh_date`, `t1`.`kill`, `t1`.`dead`, `t1`.`attack_num`, `t1`.`defend_num`, `t1`.`exp`, `t1`.`level`, `t1`.`level_up_exp`, `t1`.`health`, `t1`.`health_max`, `t1`.`damage`, `t1`.`attack_min`, `t1`.`attack_max`, `t1`.`hit_rate`, `t1`.`evade`, `t1`.`critical`, `t1`.`counter_attack`, `t1`.`energy`, `t1`.`energy_limit`, `t1`.`last_hit_time`, `t1`.`supply_gold` FROM `battle_player` AS `t1` WHERE ((`t1`.`group_id` = %s) AND (`t1`.`user_id` = %s)) LIMIT %s OFFSET %s
Params: [545423773, 435786117, 1, 0]

In [4]:
plugin.get_reply()

'未找到玩家'

In [20]:
def fix_level():
    """ 根据签到次数重置玩家等级 """
    query = Player.select()
    for player in query:
        player.level = 1
        player.level_up_exp = 100
        player.get_exp(player.num_sign * 100)
        player.save()

In [2]:
def set_supply_gold(gold):
    """ 发放补偿 """
    Player.update({Player.supply_gold: Player.supply_gold+gold}).execute()

In [3]:
# 发放1000g的补偿
# set_supply_gold(15)

In [2]:
from plugins.battle.models import sign_reply

sign_reply(user_id=435786117, group_id=542423773)

'今天已经签到过了!'

In [5]:
p = get_player(user_id=435786117, group_id=542423773)

In [10]:
print(p.status())

[CQ:at,qq=435786117]当前状态: 
生命: 250/250 体力: 20/20
攻击: 40/80
闪避\反击: 10.0%/5.0%
命中\暴击: 5.0%/10.0%
击杀\阵亡: 0/1
黄金: 628
可用属性点: 2



In [11]:
from utils import cq

### attack test

In [5]:
event={
    'group_id': 542423773,
    'message': '揍[CQ:at,qq=323566263]',
    'sender': {
        'user_id': 435786117,
        'nickname': 'LightL'
    }
}

In [6]:
from plugins.battle import AttackSomeonePlugin
plugin = AttackSomeonePlugin(event)

In [7]:
print(plugin.get_reply())

[CQ:at,qq=435786117]已经挂了。
18:29:23复活。


### test check attr

In [3]:
from plugins.battle import CheckAttributePlugin
event={
    'group_id': 542423773,
    'message': '属性',
    'sender': {
        'user_id': 435786117,
        'nickname': 'LightL'
    }
}
plugin = CheckAttributePlugin(event)

In [4]:
print(plugin.get_reply())

[CQ:at,qq=435786117]当前属性: 
等级:6
总属性/可用属性:12/0
最大生命值(10): 250
基础伤害(2): 42
小伤加成(0.0%): 42
大伤加成(0.0%): 84
闪避(0): 10.0%
反击(0): 5.0%
命中(0): 5.0%
暴击(0): 10.0%



### test add attr

In [5]:
from plugins.battle import AddAttributePlugin

event={
    'group_id': 542423773,
    'message': '加反击 -19',
    'sender': {
        'user_id': 435786117,
        'nickname': 'LightL'
    }
}
plugin = AddAttributePlugin(event)

In [7]:
print(plugin.get_reply())

In [8]:
player = get_player(user_id=435786117, group_id=542423773)
print(player.status())

[CQ:at,qq=435786117]当前状态: 
阵亡中...(18:29:23复活)
生命: 0/200 体力: 20/20
攻击: 44/88
闪避\反击: 12.0%/9.0%
命中\暴击: 5.0%/12.0%
击杀\阵亡: 0/4
黄金: 628
可用属性点: 0



## test shop

In [1]:
from plugins.battle.models import show_market

In [2]:
print(show_market())

食物:
-------
Q1面包: 体力+1 (5g)
Q5面包: 体力+5 (50g)
武器:
-------
Q1枪: 伤害+10.0% (5g)
Q5枪: 伤害+50.0% (50g)
其他:
-------
活: 立即复活 (200g)
洗点: 属性归零 (200g)
输入: 买Q1枪 <数量>
即可购买，不写数量默认买1个



In [2]:
from plugins.battle import BuyPlugin

event={
    'group_id': 542423773,
    'message': '买洗点',
    'sender': {
        'user_id': 435786117,
        'nickname': 'LightL'
    }
}
plugin = BuyPlugin(event)

In [5]:
print(plugin.get_reply())

黄金: 1228 (-200)
洗点成功，可用属性:12


In [6]:
player = get_player(user_id=435786117, group_id=542423773)
print(player.status())

[CQ:at,qq=435786117]当前状态: 
生命: 200/200 体力: 20/20
攻击: 40/80
闪避\反击: 10.0%/5.0%
命中\暴击: 5.0%/10.0%
击杀\阵亡: 0/4
黄金: 1228
可用属性点: 12



### test supply

In [1]:
from plugins.battle import SupplyPlugin

event={
    'group_id': 542423773,
    'message': '补偿',
    'sender': {
        'user_id': 435786117,
        'nickname': 'LightL'
    }
}
plugin = SupplyPlugin(event)

In [4]:
print(plugin.get_reply())

你的补偿已经领取完了！


In [1]:
from plugins.teacher.models import QuestionAnswer

In [2]:
p = QuestionAnswer.replace(question='测试的', answer='测试的2')

In [14]:
a = QuestionAnswer.update(delete=True)\
    .where(
        QuestionAnswer.question=='测试的',
        QuestionAnswer.answer=='测试的2',
    ).execute()

In [3]:
from plugins.teacher import ForgetPlugin
event = fake_group_msg_event(msg='忘记 123 123')

plugin = ForgetPlugin(event)
print(plugin.get_reply())

我没学过！


In [4]:
from peewee import SqliteDatabase
test_db = SqliteDatabase(':memory:')

In [5]:
test_db.create_tables([Player])